In [9]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    RobertaTokenizer, RobertaForSequenceClassification,
    TrainingArguments, Trainer, DataCollatorWithPadding,
    get_linear_schedule_with_warmup
)
from datasets import Dataset as HFDataset, load_dataset
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, confusion_matrix,
    classification_report, cohen_kappa_score, roc_curve, auc,
    precision_recall_curve, average_precision_score
)
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import json
from typing import Dict, List, Tuple, Optional
import logging
from datetime import datetime

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [10]:
class RobertaDataset(Dataset):
    def __init__(self, texts: List[str], labels: List[int], tokenizer, max_length: int = 512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [11]:
class RobertaTrainer:
    
    def __init__(self, model_name: str = "roberta-base", max_length: int = 512):
        self.model_name = model_name
        self.max_length = max_length
        self.tokenizer = None
        self.sentiment_model = None
        self.emotion_model = None
        self.sentiment_label_encoder = LabelEncoder()
        self.emotion_label_encoder = LabelEncoder()
        
        # Set device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {self.device}")
        
        # Create output directories
        os.makedirs("./roberta_sentiment_model", exist_ok=True)
        os.makedirs("./roberta_emotion_model", exist_ok=True)
        os.makedirs("./plots/roberta", exist_ok=True)
        os.makedirs("./results/roberta", exist_ok=True)
    
    def setup_tokenizer(self):
        logger.info("Setting up RoBERTa tokenizer...")
        self.tokenizer = RobertaTokenizer.from_pretrained(self.model_name)
        
        # Add padding token if it doesn't exist
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token
        
        logger.info("✅ Tokenizer ready!")
    
    def load_reddit_data(self, file_path: str) -> pd.DataFrame:
        logger.info(f"Loading Reddit data from {file_path}...")
        
        try:
            df = pd.read_csv(file_path)
            logger.info(f"✅ Loaded {len(df)} samples from Reddit dataset")
            
            # Check for required columns
            required_cols = ['text_content', 'sentiment', 'emotion']
            missing_cols = [col for col in required_cols if col not in df.columns]
            if missing_cols:
                raise ValueError(f"Missing required columns: {missing_cols}")
            
            # Remove rows with missing values
            df = df.dropna(subset=required_cols)
            logger.info(f"After removing missing values: {len(df)} samples")
            
            return df
            
        except Exception as e:
            logger.error(f"Error loading Reddit data: {e}")
            raise
    
    def load_external_datasets(self) -> Tuple[Dict, Dict]:
        logger.info("Loading external datasets...")
        
        # Load SST-2 for sentiment
        try:
            sst2_dataset = load_dataset("sst2")
            sentiment_data = {
                'train': sst2_dataset['train'],
                'validation': sst2_dataset['validation']
            }
            logger.info("✅ SST-2 dataset loaded for sentiment classification")
        except Exception as e:
            logger.warning(f"Could not load SST-2: {e}. Using dummy data.")
            sentiment_data = self._create_dummy_sentiment_data()
        
        # Load GoEmotions for emotion
        try:
            emotions_dataset = load_dataset("go_emotions", "simplified")
            emotion_data = {
                'train': emotions_dataset['train'],
                'validation': emotions_dataset['validation']
            }
            logger.info("✅ GoEmotions dataset loaded for emotion classification")
        except Exception as e:
            logger.warning(f"Could not load GoEmotions: {e}. Using dummy data.")
            emotion_data = self._create_dummy_emotion_data()
        
        return sentiment_data, emotion_data
    
    def _create_dummy_sentiment_data(self) -> Dict:
        dummy_texts = [
            "I love this product!", "This is terrible", "It's okay",
            "Amazing quality", "Worst experience ever", "Not bad"
        ] * 100
        dummy_labels = [1, 0, 1, 1, 0, 1] * 100
        
        dummy_data = {
            'sentence': dummy_texts,
            'label': dummy_labels
        }
        
        dataset = HFDataset.from_dict(dummy_data)
        return {'train': dataset, 'validation': dataset.select(range(100))}
    
    def _create_dummy_emotion_data(self) -> Dict:
        """Create dummy emotion data for testing"""
        dummy_texts = [
            "I'm so happy!", "This is sad", "I'm angry", "That's scary",
            "What a surprise!", "Okay", "I admire you", "That's funny"
        ] * 100
        dummy_labels = [0, 1, 2, 3, 4, 5, 6, 7] * 100
        
        dummy_data = {
            'text': dummy_texts,
            'labels': dummy_labels
        }
        
        dataset = HFDataset.from_dict(dummy_data)
        return {'train': dataset, 'validation': dataset.select(range(100))}
    
    def preprocess_external_data(self, dataset: Dict, task: str) -> Dict:
        logger.info(f"Preprocessing external data for {task}...")

        if task == 'emotion':
            # Only keep samples with label in [0, 1, 2, 3, 4, 5]
            def filter_first_6_classes(example):
                # Handle both single-label and multi-label
                if isinstance(example['labels'], list):
                    return example['labels'] and example['labels'][0] in range(6)
                else:
                    return example['labels'] in range(6)
            dataset['train'] = dataset['train'].filter(filter_first_6_classes)
            dataset['validation'] = dataset['validation'].filter(filter_first_6_classes)
        # --- END BLOCK ---

        def tokenize_function(examples):
            if task == 'sentiment':
                texts = examples['sentence']
                labels = examples['label']
            else:  # emotion
                texts = examples['text']
                # Handle multi-label to single-label conversion
                if isinstance(examples['labels'][0], list):
                    labels = [label_list[0] if label_list else 0 for label_list in examples['labels']]
                else:
                    labels = examples['labels']

            tokenized = self.tokenizer(
                texts,
                truncation=True,
                padding='max_length',
                max_length=self.max_length,
                return_tensors=None
            )

            tokenized['labels'] = labels
            return tokenized

        # Tokenize datasets
        tokenized_train = dataset['train'].map(tokenize_function, batched=True)
        tokenized_val = dataset['validation'].map(tokenize_function, batched=True)

        # Take subsets for faster training
        train_subset = tokenized_train.shuffle(seed=42).select(range(min(10000, len(tokenized_train))))
        val_subset = tokenized_val.shuffle(seed=42).select(range(min(2000, len(tokenized_val))))

        logger.info(f"✅ Preprocessed {task} data: {len(train_subset)} train, {len(val_subset)} val")

        return {'train': train_subset, 'validation': val_subset}
    
    def train_model(self, dataset: Dict, task: str, num_labels: int, 
                   learning_rate: float = 2e-5, num_epochs: int = 3,
                   batch_size: int = 16) -> Tuple[any, any]:
        logger.info(f"Training RoBERTa model for {task} classification...")
        
        # Initialize model
        model = RobertaForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=num_labels,
            ignore_mismatched_sizes=True
        )
        
        # Training arguments
        output_dir = f"./roberta_{task}_model"
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=num_epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir=f"./logs_{task}",
            logging_steps=100,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model="eval_accuracy",
            learning_rate=learning_rate,
            report_to=None,
            save_total_limit=2,
            dataloader_num_workers=0,  # Avoid multiprocessing issues
        )
        
        # Data collator
        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)
        
        # Compute metrics function
        def compute_metrics(eval_pred):
            predictions, labels = eval_pred
            predictions = np.argmax(predictions, axis=1)
            
            precision, recall, f1, _ = precision_recall_fscore_support(
                labels, predictions, average='macro', zero_division=0
            )
            accuracy = accuracy_score(labels, predictions)
            
            return {
                'accuracy': accuracy,
            }
        
        # Initialize trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=dataset['train'],
            eval_dataset=dataset['validation'],
            tokenizer=self.tokenizer,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
        )
        
        # Train
        logger.info(f"Starting {task} model training...")
        trainer.train()
        
        # Save model
        final_output_dir = f"./roberta_{task}_model_final"
        os.makedirs(final_output_dir, exist_ok=True)
        trainer.save_model(final_output_dir)
        
        logger.info(f"✅ {task.capitalize()} model training completed!")
        
        return model, trainer
    
    def prepare_reddit_data(self, df: pd.DataFrame, task: str) -> Tuple[List[str], List[int]]:
        texts = df['text_content'].tolist()
        
        if task == 'sentiment':
            labels = df['sentiment'].tolist()
            encoded_labels = self.sentiment_label_encoder.fit_transform(labels)
        else:  # emotion
            labels = df['emotion'].tolist()
            encoded_labels = self.emotion_label_encoder.fit_transform(labels)
        
        return texts, encoded_labels.tolist()
    
    def hyperparameter_tuning(self, dataset: Dict, task: str, num_labels: int) -> Dict:
        logger.info(f"Starting hyperparameter tuning for {task}...")
        
        # Define parameter grid
        param_grid = {
            'learning_rate': [1e-5, 2e-5, 3e-5, 5e-5],
            'batch_size': [8, 16, 32],
            'num_epochs': [2, 3, 4],
            'weight_decay': [0.01, 0.1]
        }
        
        best_score = 0
        best_params = {}
        results = []
        
        # Random search over parameters
        from itertools import product
        import random
        
        # Generate all combinations and sample randomly
        all_combinations = list(product(*param_grid.values()))
        random.shuffle(all_combinations)
        
        # Try up to 10 combinations
        for i, params in enumerate(all_combinations[:10]):
            param_dict = dict(zip(param_grid.keys(), params))
            logger.info(f"Trying parameters {i+1}/10: {param_dict}")
            
            try:
                # Train model with current parameters
                model, trainer = self.train_model(
                    dataset, task, num_labels,
                    learning_rate=param_dict['learning_rate'],
                    num_epochs=param_dict['num_epochs'],
                    batch_size=param_dict['batch_size']
                )
                
                # Evaluate on validation set
                eval_results = trainer.evaluate()
                score = eval_results['eval_accuracy']
                
                results.append({
                    'params': param_dict,
                    'score': score,
                    'eval_results': eval_results
                })
                
                if score > best_score:
                    best_score = score
                    best_params = param_dict
                    
                logger.info(f"Score: {score:.4f}")
                
            except Exception as e:
                logger.error(f"Error with parameters {param_dict}: {e}")
                continue
        
        logger.info(f"Best parameters for {task}: {best_params}")
        logger.info(f"Best score: {best_score:.4f}")
        
        return {
            'best_params': best_params,
            'best_score': best_score,
            'all_results': results
        }
    
    def run_complete_pipeline(self, reddit_data_path: str, 
                             perform_hyperparameter_tuning: bool = True):
        """Run the complete training and evaluation pipeline"""
        logger.info("🚀 Starting RoBERTa Complete Training Pipeline")
        logger.info("=" * 60)
        
        # Setup
        self.setup_tokenizer()
        
        # Load data
        reddit_df = self.load_reddit_data(reddit_data_path)
        sentiment_data, emotion_data = self.load_external_datasets()
        
        # Preprocess external data
        sentiment_processed = self.preprocess_external_data(sentiment_data, 'sentiment')
        emotion_processed = self.preprocess_external_data(emotion_data, 'emotion')
        
        # Prepare Reddit data
        sentiment_texts, sentiment_labels = self.prepare_reddit_data(reddit_df, 'sentiment')
        emotion_texts, emotion_labels = self.prepare_reddit_data(reddit_df, 'emotion')
        
        # Get number of classes
        n_sentiment_classes = len(self.sentiment_label_encoder.classes_)
        n_emotion_classes = len(self.emotion_label_encoder.classes_)
        
        logger.info(f"Sentiment classes: {n_sentiment_classes}")
        logger.info(f"Emotion classes: {n_emotion_classes}")
        
        results = {}
        
        # SENTIMENT TRAINING
        logger.info("\n🎯 SENTIMENT CLASSIFICATION PIPELINE")
        logger.info("=" * 50)
        
        # Pre-train on external data
        logger.info("Pre-training on SST-2 dataset...")
        sentiment_model, sentiment_trainer = self.train_model(
            sentiment_processed, 'sentiment', n_sentiment_classes
        )
        
        # Evaluate on Reddit data
        sentiment_results = self.evaluate_model(
            sentiment_model, sentiment_texts, sentiment_labels, 
            'sentiment', self.sentiment_label_encoder
        )
        
        results['sentiment'] = sentiment_results
        
        # EMOTION TRAINING
        logger.info("\nEMOTION CLASSIFICATION PIPELINE")
        logger.info("=" * 50)
        
        # Pre-train on external data
        logger.info("Pre-training on GoEmotions dataset...")
        emotion_model, emotion_trainer = self.train_model(
            emotion_processed, 'emotion', n_emotion_classes
        )
        
        # Evaluate on Reddit data
        emotion_results = self.evaluate_model(
            emotion_model, emotion_texts, emotion_labels,
            'emotion', self.emotion_label_encoder
        )

        results['emotion'] = emotion_results
        
        # FINAL SUMMARY
        logger.info("\n📋 FINAL SUMMARY")
        logger.info("=" * 50)
        
        self.create_final_summary(results)
        
        # Save all results
        self.save_results(results, 'complete_pipeline')
        
        logger.info("\nPIPELINE COMPLETED SUCCESSFULLY!")
        logger.info("📁 Output files:")
        logger.info("   - ./roberta_sentiment_model_final/ (sentiment model)")
        logger.info("   - ./roberta_emotion_model_final/ (emotion model)")
        logger.info("   - ./plots/ (all visualization plots)")
        logger.info("   - ./results/ (evaluation results)")
        
        return results
    
    def create_final_summary(self, results: Dict):
        """Create a comprehensive summary of all results"""
        summary_data = []
        
        for task in ['sentiment', 'emotion']:
            if task in results:
                res = results[task]
                summary_data.append({
                    'Task': task.capitalize(),
                    'Model': 'RoBERTa (Base)',
                    'Accuracy': f"{res['accuracy']:.4f}",
                    'Precision (Macro)': f"{res['precision_macro']:.4f}",
                    'Recall (Macro)': f"{res['recall_macro']:.4f}",
                    'F1-Score (Macro)': f"{res['f1_macro']:.4f}",
                    'F1-Score (Weighted)': f"{res['f1_weighted']:.4f}",
                    'Cohen\'s Kappa': f"{res['cohen_kappa']:.4f}"
                })
            
            # Add tuned results if available
            if f'{task}_tuned' in results:
                res = results[f'{task}_tuned']
                summary_data.append({
                    'Task': f"{task.capitalize()} (Tuned)",
                    'Model': 'RoBERTa (Tuned)',
                    'Accuracy': f"{res['accuracy']:.4f}",
                    'Precision (Macro)': f"{res['precision_macro']:.4f}",
                    'Recall (Macro)': f"{res['recall_macro']:.4f}",
                    'F1-Score (Macro)': f"{res['f1_macro']:.4f}",
                    'F1-Score (Weighted)': f"{res['f1_weighted']:.4f}",
                    'Cohen\'s Kappa': f"{res['cohen_kappa']:.4f}"
                })
        
        summary_df = pd.DataFrame(summary_data)
        print("\n" + "="*80)
        print("ROBERTA EVALUATION SUMMARY")
        print("="*80)
        print(summary_df.to_string(index=False))
        print("="*80)
        
        # Save summary
        summary_df.to_csv('results/roberta_evaluation_summary.csv', index=False)
        logger.info("Summary saved to: results/roberta_evaluation_summary.csv")


In [12]:
trainer = RobertaTrainer()
trainer.setup_tokenizer()

INFO:__main__:Using device: cuda
INFO:__main__:Setting up RoBERTa tokenizer...
INFO:__main__:✅ Tokenizer ready!


In [13]:
reddit_df = trainer.load_reddit_data('annotated_reddit_posts.csv')
reddit_df.head()

INFO:__main__:Loading Reddit data from annotated_reddit_posts.csv...
INFO:__main__:✅ Loaded 95 samples from Reddit dataset
INFO:__main__:After removing missing values: 95 samples


,id,text_content,original_text,type,score,subjectivity,sentiment,emotion
0,d8kzu3m,ya screw username really looking forward note ...,Ya this screws me over completely. I was reall...,comment,2,0.464167,Negative,Sadness
1,5jd0fx,username samsung galaxy note7 still more user ...,Cancelled Samsung Galaxy Note7 still has more ...,post,95,0.500000,Neutral,Surprise
2,dbfyq3r,traded note 7 s7 edge really hope samsung user...,I traded my Note 7 in for an S7 Edge. I reall...,comment,9,0.414286,Negative,Sadness
3,dbgcdgl,reading username report battery design failed ...,From reading the independent report of why its...,comment,2,0.406250,Positive,Sadness
4,dbftpbx,maybe phone unique username feature explode us...,Maybe the phone's unique exploding feature (or...,comment,-4,0.800000,Negative,Anger


In [16]:
sentiment_data, emotion_data = trainer.load_external_datasets()

INFO:__main__:Loading external datasets...
INFO:__main__:✅ SST-2 dataset loaded for sentiment classification
INFO:__main__:✅ GoEmotions dataset loaded for emotion classification


In [17]:
sentiment_processed = trainer.preprocess_external_data(sentiment_data, 'sentiment')
emotion_processed = trainer.preprocess_external_data(emotion_data, 'emotion')

INFO:__main__:Preprocessing external data for sentiment...
Map: 100%|██████████| 872/872 [00:00<00:00, 3665.91 examples/s]
INFO:__main__:✅ Preprocessed sentiment data: 10000 train, 872 val
INFO:__main__:Preprocessing external data for emotion...
Map: 100%|██████████| 1717/1717 [00:00<00:00, 4889.51 examples/s]
INFO:__main__:✅ Preprocessed emotion data: 10000 train, 1717 val


In [18]:
sentiment_texts, sentiment_labels = trainer.prepare_reddit_data(reddit_df, 'sentiment')
emotion_texts, emotion_labels = trainer.prepare_reddit_data(reddit_df, 'emotion')
n_sentiment_classes = len(trainer.sentiment_label_encoder.classes_)
n_emotion_classes = len(trainer.emotion_label_encoder.classes_)

##### Save label encoders

In [20]:
import joblib

# Save the label encoders
joblib.dump(trainer.sentiment_label_encoder, 'sentiment_label_encoder.pkl')
joblib.dump(trainer.emotion_label_encoder, 'emotion_label_encoder.pkl')

print("Label encoders saved!")
print("Sentiment classes:", trainer.sentiment_label_encoder.classes_)
print("Emotion classes:", trainer.emotion_label_encoder.classes_)

Label encoders saved!
Sentiment classes: ['Negative' 'Neutral' 'Positive']
Emotion classes: ['Anger' 'Fear' 'Joy' 'No Emotion' 'Sadness' 'Surprise']


### Prep for evaluation

In [3]:
import joblib

# Load the label encoder
sentiment_label_encoder = joblib.load('enc/sentiment_label_encoder.pkl')
emotion_label_encoder = joblib.load('enc/emotion_label_encoder.pkl')

In [11]:
# Extract texts and true labels
test_texts = reddit_df['text_content'].tolist()

# Encode true labels using the same encoders used during training
sentiment_true_labels = sentiment_label_encoder.transform(reddit_df['sentiment'].tolist())
emotion_true_labels = emotion_label_encoder.transform(reddit_df['emotion'].tolist())

print(f"Sentiment classes: {sentiment_label_encoder.classes_}")
print(f"Emotion classes: {emotion_label_encoder.classes_}")

Sentiment classes: ['Negative' 'Neutral' 'Positive']
Emotion classes: ['Anger' 'Fear' 'Joy' 'No Emotion' 'Sadness' 'Surprise']


In [14]:
import pandas as pd
import numpy as np
import torch
import joblib
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm

def evaluate_roberta_model(model_dir, label_encoder_path, csv_file, target_column, model_name):
    
    print(f"🚀 Starting evaluation of {model_name} model...")
    
    # 1. Load model, tokenizer, and label encoder
    print("📂 Loading model and tokenizer...")
    tokenizer = RobertaTokenizer.from_pretrained(model_dir)
    model = RobertaForSequenceClassification.from_pretrained(model_dir)
    label_encoder = joblib.load(label_encoder_path)
    
    # 2. Set device and move model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"🖥️ Using device: {device}")
    model = model.to(device)
    model.eval()
    
    # 3. Load and prepare data
    print("📊 Loading test data...")
    df = pd.read_csv(csv_file)
    test_texts = df['text_content'].tolist()
    true_labels_text = df[target_column].tolist()
    
    # Encode true labels
    true_labels_encoded = label_encoder.transform(true_labels_text)
    
    print(f"📋 Dataset info:")
    print(f"   - Total samples: {len(test_texts)}")
    print(f"   - Classes: {list(label_encoder.classes_)}")
    print(f"   - Class distribution:")
    for class_name in label_encoder.classes_:
        count = sum(1 for label in true_labels_text if label == class_name)
        print(f"     {class_name}: {count}")
    
    # 4. Run evaluation
    print("🔮 Running model evaluation...")
    
    predicted_labels_encoded = []
    
    for text in tqdm(test_texts, desc="Predicting"):
        # Tokenize
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=512
        )
        
        # Move to device
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Predict
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class_id = logits.argmax(dim=-1).item()
        
        predicted_labels_encoded.append(predicted_class_id)
    
    # 5. Calculate metrics
    print("📈 Calculating metrics...")
    
    accuracy = accuracy_score(true_labels_encoded, predicted_labels_encoded)
    precision, recall, f1_macro, _ = precision_recall_fscore_support(
        true_labels_encoded, predicted_labels_encoded, average='macro'
    )
    _, _, f1_weighted, _ = precision_recall_fscore_support(
        true_labels_encoded, predicted_labels_encoded, average='weighted'
    )
    
    # 6. Prepare results
    results = {
        'model_name': model_name,
        'target_column': target_column,
        'accuracy': accuracy,
        'f1_macro': f1_macro,
        'predictions': predicted_labels_encoded,
        'true_labels': true_labels_encoded,
        'class_names': list(label_encoder.classes_)
    }
    
    # 7. Print results
    print(f"\n{model_name} Model Evaluation Results:")
    print(f"{'='*50}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"F1-Score (Macro): {f1_macro:.4f}")
    
    return results

In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

class RedditTextPreprocessor:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
        # Remove some emotion-related words from stopwords to preserve sentiment
        emotion_words = {
            'not', 'no', 'nor', 'but', 'however', 'although', 'though',
            'very', 'really', 'quite', 'too', 'so', 'more', 'most',
            'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other',
            'some', 'such', 'only', 'own', 'same', 'than', 'too', 'very'
        }
        self.stop_words = self.stop_words - emotion_words
    
    def remove_urls(self, text):
        text = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        text = re.sub(r'www\.(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text)
        text = re.sub(r'/r/[A-Za-z0-9_]+', '', text)
        return text
    
    def remove_mentions_hashtags(self, text):
        text = re.sub(r'@[A-Za-z0-9_]+', '', text)
        text = re.sub(r'#([A-Za-z0-9_]+)', r'\1', text)
        return text
    
    def remove_html_tags(self, text):
        text = re.sub(r'<[^>]+>', '', text)
        return text
    
    def handle_reddit_formatting(self, text):
        text = re.sub(r'\[([^\]]+)\]\([^\)]+\)', r'\1', text)
        text = re.sub(r'u/[A-Za-z0-9_]+', '', text)
        text = re.sub(r'\*\*([^\*]+)\*\*', r'\1', text)
        text = re.sub(r'\*([^\*]+)\*', r'\1', text)
        text = re.sub(r'^>', '', text, flags=re.MULTILINE)
        return text
    
    def convert_emojis(self, text):
        try:
            import emoji
            text = emoji.demojize(text, delimiters=(" ", " "))
            text = re.sub(r':[a-zA-Z_]+:', lambda m: m.group().replace('_', ' ').replace(':', ''), text)
        except ImportError:
            pass
        return text
    
    def clean_special_characters(self, text):
        text = re.sub(r'!+', ' EXCLAMATION ', text)
        text = re.sub(r'\?+', ' QUESTION ', text)
        text = re.sub(r'[^\w\s\']', ' ', text)
        text = text.replace(' EXCLAMATION ', ' ! ')
        text = text.replace(' QUESTION ', ' ? ')
        text = re.sub(r"'", '', text)
        return text
    
    def normalize_whitespace(self, text):
        text = re.sub(r'\s+', ' ', text)
        text = text.strip()
        return text
    
    def remove_stopwords(self, text):
        try:
            words = word_tokenize(text.lower())
            filtered_words = [word for word in words if word not in self.stop_words]
            return ' '.join(filtered_words)
        except:
            words = text.lower().split()
            filtered_words = [word for word in words if word not in self.stop_words]
            return ' '.join(filtered_words)
    
    def lemmatize_text(self, text):
        try:
            words = word_tokenize(text)
            lemmatized_words = [self.lemmatizer.lemmatize(word) for word in words]
            return ' '.join(lemmatized_words)
        except:
            words = text.split()
            lemmatized_words = [self.lemmatizer.lemmatize(word) for word in words]
            return ' '.join(lemmatized_words)
    
    def anonymize_identifiers(self, text):
        text = re.sub(r'\b\d{3}[-.]?\d{3}[-.]?\d{4}\b', '[PHONE]', text)
        text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '[EMAIL]', text)
        text = re.sub(r'\b[A-Za-z0-9_]{8,}\b', '[USERNAME]', text)
        return text
    
    def preprocess_text(self, text):
        if not isinstance(text, str):
            return ""
        
        text = self.handle_reddit_formatting(text)
        text = self.remove_urls(text)
        text = self.remove_mentions_hashtags(text)
        text = self.remove_html_tags(text)
        text = self.convert_emojis(text)
        text = self.anonymize_identifiers(text)
        text = self.clean_special_characters(text)
        text = self.normalize_whitespace(text)
        text = text.lower()
        text = self.remove_stopwords(text)
        text = self.lemmatize_text(text)
        text = self.normalize_whitespace(text)
        
        return text

preprocessor = RedditTextPreprocessor()

### HP Tuning

In [4]:
import optuna
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    RobertaTokenizer, RobertaForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import logging
import os
from datetime import datetime

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class RobertaDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

class HyperparameterTuner:
    def __init__(self, data_path, task_type='emotion', n_trials=50, cv_folds=3):
        self.data_path = data_path
        self.task_type = task_type
        self.n_trials = n_trials
        self.cv_folds = cv_folds
        
        # Load and prepare data
        self.df = pd.read_csv(data_path)
        self.texts = self.df['text_content'].tolist()
        self.labels_text = self.df[task_type].tolist()
        
        # Encode labels
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels_text)
        self.n_classes = len(self.label_encoder.classes_)
        
        logger.info(f"Loaded {len(self.texts)} samples for {task_type} task")
        logger.info(f"Classes: {list(self.label_encoder.classes_)}")
        logger.info(f"Number of classes: {self.n_classes}")
        
        # Initialize tokenizer
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        
        # Create output directory
        self.output_dir = f"hyperparameter_tuning_{task_type}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        os.makedirs(self.output_dir, exist_ok=True)
    
    def objective(self, trial):
        # Sample hyperparameters
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 5e-4, log=True)
        batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])
        dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
        num_epochs = trial.suggest_int('num_epochs', 3, 8)
        warmup_ratio = trial.suggest_float('warmup_ratio', 0.05, 0.2)
        weight_decay = trial.suggest_float('weight_decay', 0.01, 0.3)
        
        logger.info(f"Trial {trial.number}: lr={learning_rate:.2e}, bs={batch_size}, "
                   f"dropout={dropout_rate:.3f}, epochs={num_epochs}")
        
        # Cross-validation scores
        cv_scores = []
        
        # Stratified K-Fold
        skf = StratifiedKFold(n_splits=self.cv_folds, shuffle=True, random_state=42)
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(self.texts, self.labels)):
            logger.info(f"  Fold {fold + 1}/{self.cv_folds}")
            
            # Split data
            train_texts = [self.texts[i] for i in train_idx]
            train_labels = [self.labels[i] for i in train_idx]
            val_texts = [self.texts[i] for i in val_idx]
            val_labels = [self.labels[i] for i in val_idx]
            
            # Create datasets
            train_dataset = RobertaDataset(train_texts, train_labels, self.tokenizer)
            val_dataset = RobertaDataset(val_texts, val_labels, self.tokenizer)
            
            # Initialize model with custom dropout
            model = RobertaForSequenceClassification.from_pretrained(
                'roberta-base',
                num_labels=self.n_classes,
                hidden_dropout_prob=dropout_rate,
                attention_probs_dropout_prob=dropout_rate
            )
            
            # Training arguments
            training_args = TrainingArguments(
                output_dir=f"{self.output_dir}/trial_{trial.number}_fold_{fold}",
                num_train_epochs=num_epochs,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                learning_rate=learning_rate,
                weight_decay=weight_decay,
                warmup_ratio=warmup_ratio,
                logging_dir=f"{self.output_dir}/logs",
                logging_steps=50,
                evaluation_strategy="epoch",
                save_strategy="epoch",
                load_best_model_at_end=True,
                metric_for_best_model="eval_f1",
                greater_is_better=True,
                remove_unused_columns=False,
                dataloader_num_workers=0,
                report_to="none",  # Disable wandb/tensorboard
                save_total_limit=1,  # Only keep best checkpoint
            )
            
            # Custom compute metrics function
            def compute_metrics(eval_pred):
                predictions, labels = eval_pred
                predictions = np.argmax(predictions, axis=1)
                
                if self.task_type == 'emotion':
                    # Weighted F1 for emotion (imbalanced classes)
                    f1 = f1_score(labels, predictions, average='weighted')
                else:
                    # Macro F1 for sentiment
                    f1 = f1_score(labels, predictions, average='macro')
                
                return {'f1': f1}
            
            # Initialize trainer
            trainer = Trainer(
                model=model,
                args=training_args,
                train_dataset=train_dataset,
                eval_dataset=val_dataset,
                compute_metrics=compute_metrics,
                callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
            )
            
            # Train model
            try:
                trainer.train()
                
                # Evaluate
                eval_results = trainer.evaluate()
                fold_score = eval_results['eval_f1']
                cv_scores.append(fold_score)
                
                logger.info(f"    Fold {fold + 1} F1: {fold_score:.4f}")
                
            except Exception as e:
                logger.error(f"    Fold {fold + 1} failed: {str(e)}")
                cv_scores.append(0.0)  # Penalty for failed runs
            
            # Clean up to save memory
            del model, trainer
            torch.cuda.empty_cache()
        
        # Calculate mean CV score
        mean_score = np.mean(cv_scores)
        std_score = np.std(cv_scores)
        
        logger.info(f"Trial {trial.number} completed: Mean F1 = {mean_score:.4f} ± {std_score:.4f}")
        
        return mean_score
    
    def tune(self):
        logger.info(f"Starting hyperparameter tuning for {self.task_type} task")
        logger.info(f"Search space:")
        logger.info(f"  - Learning rate: [1e-5, 5e-4]")
        logger.info(f"  - Batch size: [8, 16, 32]")
        logger.info(f"  - Dropout rate: [0.1, 0.5]")
        logger.info(f"  - Epochs: [3, 8]")
        logger.info(f"  - Warmup ratio: [0.05, 0.2]")
        logger.info(f"  - Weight decay: [0.01, 0.3]")
        
        # Create study
        study = optuna.create_study(
            direction='maximize',
            sampler=optuna.samplers.TPESampler(seed=42),
            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=10)
        )
        
        # Optimize
        study.optimize(self.objective, n_trials=self.n_trials)
        
        # Save results
        self.save_results(study)
        
        return study
    
    def save_results(self, study):
        logger.info("Saving optimization results...")
        
        # Best parameters
        best_params = study.best_params
        best_score = study.best_value
        
        logger.info(f"Best score: {best_score:.4f}")
        logger.info(f"Best parameters: {best_params}")
        
        # Save to file
        results = {
            'task_type': self.task_type,
            'best_score': best_score,
            'best_params': best_params,
            'n_trials': len(study.trials),
            'cv_folds': self.cv_folds
        }
        
        import json
        with open(f"{self.output_dir}/best_hyperparameters.json", 'w') as f:
            json.dump(results, f, indent=2)
        
        # Save study
        import pickle
        with open(f"{self.output_dir}/optuna_study.pkl", 'wb') as f:
            pickle.dump(study, f)
        
        # Create trials dataframe
        trials_df = study.trials_dataframe()
        trials_df.to_csv(f"{self.output_dir}/all_trials.csv", index=False)
        
        logger.info(f"Results saved to {self.output_dir}/")
        
        return results

def train_final_model(best_params, data_path, task_type, output_dir):
    logger.info(f"Training final {task_type} model with best hyperparameters...")
    
    # Load data
    df = pd.read_csv(data_path)
    texts = df['text_content'].tolist()
    labels_text = df[task_type].tolist()
    
    # Encode labels
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels_text)
    n_classes = len(label_encoder.classes_)
    
    # Save label encoder
    import joblib
    os.makedirs(f"{output_dir}/enc", exist_ok=True)
    joblib.dump(label_encoder, f"{output_dir}/enc/{task_type}_label_encoder.pkl")
    
    # Initialize tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    
    # Create dataset
    dataset = RobertaDataset(texts, labels, tokenizer)
    
    # Initialize model
    model = RobertaForSequenceClassification.from_pretrained(
        'roberta-base',
        num_labels=n_classes,
        hidden_dropout_prob=best_params['dropout_rate'],
        attention_probs_dropout_prob=best_params['dropout_rate']
    )
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=best_params['num_epochs'],
        per_device_train_batch_size=best_params['batch_size'],
        per_device_eval_batch_size=best_params['batch_size'],
        learning_rate=best_params['learning_rate'],
        weight_decay=best_params['weight_decay'],
        warmup_ratio=best_params['warmup_ratio'],
        logging_dir=f"{output_dir}/logs",
        logging_steps=50,
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_num_workers=0,
        report_to="none"
    )
    
    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer
    )
    
    # Train
    trainer.train()
    
    # Save final model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    
    logger.info(f"Final {task_type} model saved to {output_dir}")
    
    return model, tokenizer, label_encoder

c:\Users\hankaixin\Desktop\multitask\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Train sentiment model

In [21]:
sentiment_model, sentiment_trainer = trainer.train_model(
    sentiment_processed, 'sentiment', n_sentiment_classes
)

INFO:__main__:Training RoBERTa model for sentiment classification...
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:🚀 Starting sentiment model training...
  5%|▌         | 100/1875 [26:14<6:55:36, 14.05s/it]

{'loss': 1.045, 'grad_norm': 5.94059419631958, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.16}


 11%|█         | 200/1875 [57:25<6:38:38, 14.28s/it] 

{'loss': 0.4976, 'grad_norm': 11.920705795288086, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.32}


 16%|█▌        | 300/1875 [1:25:55<22:36:17, 51.67s/it]

...

                                                      
100%|██████████| 1875/1875 [11:36:35<00:00, 15.29s/it]

{'eval_loss': 0.31552377343177795, 'eval_accuracy': 0.9288990825688074, 'eval_f1': 0.9288451819407009, 'eval_precision': 0.9291403286978508, 'eval_recall': 0.9287067441273049, 'eval_runtime': 238.9574, 'eval_samples_per_second': 3.649, 'eval_steps_per_second': 0.23, 'epoch': 3.0}


100%|██████████| 1875/1875 [11:36:37<00:00, 22.29s/it]


{'train_runtime': 41797.1955, 'train_samples_per_second': 0.718, 'train_steps_per_second': 0.045, 'train_loss': 0.28538743743896483, 'epoch': 3.0}


INFO:__main__:✅ Sentiment model training completed!


In [ ]:
# Save model and tokenizer to a directory
save_directory = "./roberta_sentiment_model"
sentiment_model.save_pretrained(save_directory)
trainer.tokenizer.save_pretrained(save_directory) 

('./roberta_sentiment_model\\tokenizer_config.json',
 './roberta_sentiment_model\\special_tokens_map.json',
 './roberta_sentiment_model\\vocab.json',
 './roberta_sentiment_model\\merges.txt',
 './roberta_sentiment_model\\added_tokens.json')


### Load model

In [5]:
sentiment_model_dir = "./roberta_sentiment_model"
sentiment_tokenizer = RobertaTokenizer.from_pretrained(sentiment_model_dir)
sentiment_model = RobertaForSequenceClassification.from_pretrained(sentiment_model_dir)
sentiment_label_encoder = joblib.load('enc/sentiment_label_encoder.pkl')

# Set device and move models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

emotion_model = sentiment_model.to(device)
emotion_model.eval()

print("✅ Models moved to device and set to eval mode!")
print("Available sentiment classes:", sentiment_label_encoder.classes_)

Using device: cuda
✅ Models moved to device and set to eval mode!
Available sentiment classes: ['Negative' 'Neutral' 'Positive']


### Evaluate model

In [15]:
# Example usage
sentiment_results = evaluate_roberta_model(
    model_dir="./roberta_sentiment_model",
    label_encoder_path="enc/sentiment_label_encoder.pkl", 
    csv_file="annotated_reddit_posts.csv",
    target_column="sentiment",
    model_name="Sentiment"
)

🚀 Starting evaluation of Sentiment model...
📂 Loading model and tokenizer...
🖥️ Using device: cuda
📊 Loading test data...
📋 Dataset info:
   - Total samples: 95
   - Classes: [np.str_('Negative'), np.str_('Neutral'), np.str_('Positive')]
   - Class distribution:
     Negative: 52
     Neutral: 21
     Positive: 22
🔮 Running model evaluation...


Predicting: 100%|██████████| 95/95 [00:01<00:00, 61.69it/s]


📈 Calculating metrics...

Sentiment Model Evaluation Results:
Accuracy: 0.5579
F1-Score (Macro): 0.3095


#### Predict with sentiment

In [45]:
def predict_sentiment(text):

    # Preprocess the input text
    preprocessed_text = preprocessor.preprocess_text(text)
    
    # Handle empty text after preprocessing
    if not preprocessed_text or len(preprocessed_text.strip()) == 0:
        return {
            'predicted_label': 'Neutral',  # Default fallback
            'confidence': 0.0,
            'class_id': -1,
            'all_probabilities': [],
            'preprocessed_text': preprocessed_text,
            'error': 'Text became empty after preprocessing'
        }
    
    # Tokenize the preprocessed text
    inputs = sentiment_tokenizer(
        preprocessed_text,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    )
    
    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Run inference
    with torch.no_grad():
        outputs = sentiment_model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class_id = logits.argmax(dim=-1).item()
    
    # Map back to original label
    predicted_label = sentiment_label_encoder.inverse_transform([predicted_class_id])[0]
    confidence = probabilities[0][predicted_class_id].item()
    
    return {
        'predicted_label': predicted_label,
        'confidence': confidence,
        'class_id': predicted_class_id,
        'all_probabilities': probabilities[0].cpu().numpy(),
        'preprocessed_text': preprocessed_text,
        'original_text': text
    }

test_cases = [
    "I absolutely love this product! 😍 It's amazing!",
    "This is terrible... I hate it so much 😠",
    "Check out https://example.com and @username for more info #awesome",
    "My phone just **exploded** while charging! WTF Samsung?!",
    "Meh, it's okay I guess. Nothing special.",
]

print("\n😊 SENTIMENT PREDICTIONS:")
print("="*60)
for text in test_cases:
    result = predict_sentiment(text)
    print(f"Original: {text}")
    print(f"Preprocessed: {result['preprocessed_text']}")
    print(f"Sentiment: {result['predicted_label']} (confidence: {result['confidence']:.4f})")
    print("-" * 40)


😊 SENTIMENT PREDICTIONS:
Original: I absolutely love this product! 😍 It's amazing!
Preprocessed: username love product ! username eye amazing !
Sentiment: Positive (confidence: 0.3398)
----------------------------------------
Original: This is terrible... I hate it so much 😠
Preprocessed: username hate so much username
Sentiment: Negative (confidence: 0.4460)
----------------------------------------
Original: Check out https://example.com and @username for more info #awesome
Preprocessed: check more info awesome
Sentiment: Negative (confidence: 0.3816)
----------------------------------------
Original: My phone just **exploded** while charging! WTF Samsung?!
Preprocessed: phone username username ! wtf samsung ? !
Sentiment: Negative (confidence: 0.4850)
----------------------------------------
Original: Meh, it's okay I guess. Nothing special.
Preprocessed: meh okay guess nothing special
Sentiment: Negative (confidence: 0.3695)
----------------------------------------


### HP Tuning

In [43]:
# 1. Create the tuner for the sentiment task
sentiment_tuner = HyperparameterTuner(
    data_path="annotated_reddit_posts.csv", 
    task_type="sentiment",                  
    n_trials=30,                           
    cv_folds=3                             
)

# 2. Run the hyperparameter search
sentiment_study = sentiment_tuner.tune()

INFO:__main__:Loaded 95 samples for sentiment task
INFO:__main__:Classes: [np.str_('Negative'), np.str_('Neutral'), np.str_('Positive')]
INFO:__main__:Number of classes: 3
INFO:__main__:Starting hyperparameter tuning for sentiment task
INFO:__main__:Search space:
INFO:__main__:  - Learning rate: [1e-5, 5e-4]
INFO:__main__:  - Batch size: [8, 16, 32]
INFO:__main__:  - Dropout rate: [0.1, 0.5]
INFO:__main__:  - Epochs: [3, 8]
INFO:__main__:  - Warmup ratio: [0.05, 0.2]
INFO:__main__:  - Weight decay: [0.01, 0.3]
[I 2025-07-09 12:31:25,551] A new study created in memory with name: no-name-3d802fcd-c6d8-4d63-a4cb-dd568b7c6182
INFO:__main__:Trial 0: lr=4.33e-05, bs=8, dropout=0.162, epochs=3
INFO:__main__:  Fold 1/3
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN thi

{'eval_loss': 1.0000598430633545, 'eval_f1': 0.24, 'eval_runtime': 0.5308, 'eval_samples_per_second': 60.29, 'eval_steps_per_second': 7.536, 'epoch': 1.0}


                                               
 67%|██████▋   | 16/24 [00:09<00:03,  2.19it/s]

{'eval_loss': 0.9987433552742004, 'eval_f1': 0.24, 'eval_runtime': 0.5242, 'eval_samples_per_second': 61.051, 'eval_steps_per_second': 7.631, 'epoch': 2.0}


                                               
100%|██████████| 24/24 [00:15<00:00,  2.18it/s]

...

100%|██████████| 12/12 [00:25<00:00,  1.18s/it]

{'eval_loss': 0.9979338645935059, 'eval_f1': 0.23611111111111113, 'eval_runtime': 0.7114, 'eval_samples_per_second': 43.579, 'eval_steps_per_second': 2.812, 'epoch': 3.0}


100%|██████████| 12/12 [00:29<00:00,  1.18s/it]

{'train_runtime': 29.8524, 'train_samples_per_second': 6.432, 'train_steps_per_second': 0.402, 'train_loss': 1.0628913243611653, 'epoch': 3.0}


100%|██████████| 2/2 [00:00<00:00,  8.81it/s]
INFO:__main__:    Fold 3 F1: 0.2361
INFO:__main__:Trial 29 completed: Mean F1 = 0.2358 ± 0.0036
[I 2025-07-09 15:29:02,763] Trial 29 finished with value: 0.23580120937263796 and parameters: {'learning_rate': 0.0001550017201595809, 'batch_size': 16, 'dropout_rate': 0.2716771675719538, 'num_epochs': 3, 'warmup_ratio': 0.13980717802070614, 'weight_decay': 0.19358119823733788}. Best is trial 4 with value: 0.35057139664982806.
INFO:__main__:Saving optimization results...
INFO:__main__:Best score: 0.3506
INFO:__main__:Best parameters: {'learning_rate': 1.289795048085554e-05, 'batch_size': 16, 'dropout_rate': 0.2218455076693483, 'num_epochs': 3, 'warmup_ratio': 0.15263495397682356, 'weight_decay': 0.13764422318448438}
INFO:__main__:Results saved to hyperparameter_tuning_sentiment_20250709_123125/


In [44]:
# Train final sentiment model
sentiment_best_params = sentiment_study.best_params
sentiment_model, sentiment_tokenizer, sentiment_encoder = train_final_model(
    sentiment_best_params,
    "annotated_reddit_posts.csv", 
    "sentiment",
    "./roberta_sentiment_model_optimized"
)

INFO:__main__:Training final sentiment model with best hyperparameters...
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 18/18 [00:23<00:00,  1.32s/it]


{'train_runtime': 23.7921, 'train_samples_per_second': 11.979, 'train_steps_per_second': 0.757, 'train_loss': 1.0695559183756511, 'epoch': 3.0}


INFO:__main__:Final sentiment model saved to ./roberta_sentiment_model_optimized


In [47]:
print(sentiment_best_params)

{'learning_rate': 1.289795048085554e-05, 'batch_size': 16, 'dropout_rate': 0.2218455076693483, 'num_epochs': 3, 'warmup_ratio': 0.15263495397682356, 'weight_decay': 0.13764422318448438}


In [16]:
# Evaluate Sentiment Model
print("Evaluating Sentiment Model...")
sentiment_results = evaluate_roberta_model(
    model_dir="./roberta_sentiment_model_optimized",
    label_encoder_path="enc/sentiment_label_encoder.pkl",
    csv_file="annotated_reddit_posts.csv",
    target_column="sentiment",
    model_name="Sentiment"
)

Evaluating Sentiment Model...
🚀 Starting evaluation of Sentiment model...
📂 Loading model and tokenizer...
🖥️ Using device: cuda
📊 Loading test data...
📋 Dataset info:
   - Total samples: 95
   - Classes: [np.str_('Negative'), np.str_('Neutral'), np.str_('Positive')]
   - Class distribution:
     Negative: 52
     Neutral: 21
     Positive: 22
🔮 Running model evaluation...


Predicting: 100%|██████████| 95/95 [00:01<00:00, 53.11it/s]

📈 Calculating metrics...

Sentiment Model Evaluation Results:
Accuracy: 0.5474
F1-Score (Macro): 0.2358


# Train emotion model

In [9]:
n_emotion_classes = len(set(emotion_labels))
emotion_model, emotion_trainer = trainer.train_model(
    emotion_processed, 'emotion', n_emotion_classes
)

INFO:__main__:Training RoBERTa model for emotion classification...
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
INFO:__main__:🚀 Starting emotion model training...
  5%|▌         | 100/1875 [04:10<1:14:52,  2.53s/it]

{'loss': 1.7369, 'grad_norm': 4.5536675453186035, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.16}


 11%|█         | 200/1875 [05:34<23:27,  1.19it/s]  

{'loss': 1.6319, 'grad_norm': 10.80386734008789, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.32}


 16%|█▌        | 300/1875 [06:57<21:18,  1.23it/s]

...

                                                   
100%|██████████| 1875/1875 [29:40<00:00,  1.24it/s]

{'eval_loss': 0.6570369005203247, 'eval_accuracy': 0.7740244612696564, 'eval_f1': 0.7510951876420294, 'eval_precision': 0.7517022306451588, 'eval_recall': 0.7521308515086198, 'eval_runtime': 26.5048, 'eval_samples_per_second': 64.781, 'eval_steps_per_second': 4.075, 'epoch': 3.0}


100%|██████████| 1875/1875 [29:41<00:00,  1.05it/s]


{'train_runtime': 1781.9952, 'train_samples_per_second': 16.835, 'train_steps_per_second': 1.052, 'train_loss': 0.745460292561849, 'epoch': 3.0}


INFO:__main__:✅ Emotion model training completed!


In [ ]:
# Save model and tokenizer to a directory
save_directory = "./roberta_emotion_model"
sentiment_model.save_pretrained(save_directory)
trainer.tokenizer.save_pretrained(save_directory) 

### Load model

In [17]:
emotion_model_dir = "./roberta_emotion_model"
emotion_tokenizer = RobertaTokenizer.from_pretrained(emotion_model_dir)
emotion_model = RobertaForSequenceClassification.from_pretrained(emotion_model_dir)
emotion_label_encoder = joblib.load('enc/emotion_label_encoder.pkl')

# Set device and move models
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

emotion_model = emotion_model.to(device)
emotion_model.eval()

print("✅ Models moved to device and set to eval mode!")
print("Available emotion classes:", emotion_label_encoder.classes_)

Using device: cuda
✅ Models moved to device and set to eval mode!
Available emotion classes: ['Anger' 'Fear' 'Joy' 'No Emotion' 'Sadness' 'Surprise']


#### Evaluate model

In [18]:
# Evaluate Emotion Model
print("🎭 Evaluating Emotion Model...")
emotion_results = evaluate_roberta_model(
    model_dir="./roberta_emotion_model",
    label_encoder_path="enc/emotion_label_encoder.pkl",
    csv_file="annotated_reddit_posts.csv",
    target_column="emotion",
    model_name="Emotion"
)


🎭 Evaluating Emotion Model...
🚀 Starting evaluation of Emotion model...
📂 Loading model and tokenizer...
🖥️ Using device: cuda
📊 Loading test data...
📋 Dataset info:
   - Total samples: 95
   - Classes: [np.str_('Anger'), np.str_('Fear'), np.str_('Joy'), np.str_('No Emotion'), np.str_('Sadness'), np.str_('Surprise')]
   - Class distribution:
     Anger: 24
     Fear: 4
     Joy: 12
     No Emotion: 22
     Sadness: 21
     Surprise: 12
🔮 Running model evaluation...


Predicting: 100%|██████████| 95/95 [00:01<00:00, 54.55it/s]

📈 Calculating metrics...

Emotion Model Evaluation Results:
Accuracy: 0.1895
F1-Score (Macro): 0.1106


#### Predict with emotion

In [37]:
def predict_emotion(text):
    preprocessed_text = preprocessor.preprocess_text(text)
    
    # Handle empty text after preprocessing
    if not preprocessed_text or len(preprocessed_text.strip()) == 0:
        return {
            'predicted_label': 'No Emotion',  # Default fallback
            'confidence': 0.0,
            'class_id': -1,
            'all_probabilities': [],
            'preprocessed_text': preprocessed_text,
            'error': 'Text became empty after preprocessing'
        }
    
    # Tokenize the preprocessed text
    inputs = emotion_tokenizer(
        preprocessed_text,  # Use preprocessed text
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    )
    
    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Run inference
    with torch.no_grad():
        outputs = emotion_model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class_id = logits.argmax(dim=-1).item()
    
    # Map back to original label
    predicted_label = emotion_label_encoder.inverse_transform([predicted_class_id])[0]
    confidence = probabilities[0][predicted_class_id].item()
    
    return {
        'predicted_label': predicted_label,
        'confidence': confidence,
        'class_id': predicted_class_id,
        'all_probabilities': probabilities[0].cpu().numpy(),
        'preprocessed_text': preprocessed_text,  # Show what the model actually saw
        'original_text': text  # Keep original for reference
    }

# Test with various types of input
test_cases = [
    "I absolutely love this product! 😍 It's amazing!",
    "This is terrible... I hate it so much 😠",
    "Check out https://example.com and @username for more info #awesome",
    "My phone just **exploded** while charging! WTF Samsung?!",
    "Meh, it's okay I guess. Nothing special.",
]

print("🎭 EMOTION PREDICTIONS:")
print("="*60)
for text in test_cases:
    result = predict_emotion(text)
    print(f"Original: {text}")
    print(f"Preprocessed: {result['preprocessed_text']}")
    print(f"Emotion: {result['predicted_label']} (confidence: {result['confidence']:.4f})")
    print("-" * 40)

🎭 EMOTION PREDICTIONS:
Original: I absolutely love this product! 😍 It's amazing!
Preprocessed: username love product ! username eye amazing !
Emotion: Fear (confidence: 0.9986)
----------------------------------------
Original: This is terrible... I hate it so much 😠
Preprocessed: username hate so much username
Emotion: Anger (confidence: 0.9975)
----------------------------------------
Original: Check out https://example.com and @username for more info #awesome
Preprocessed: check more info awesome
Emotion: Fear (confidence: 0.9981)
----------------------------------------
Original: My phone just **exploded** while charging! WTF Samsung?!
Preprocessed: phone username username ! wtf samsung ? !
Emotion: Anger (confidence: 0.9976)
----------------------------------------
Original: Meh, it's okay I guess. Nothing special.
Preprocessed: meh okay guess nothing special
Emotion: Anger (confidence: 0.9977)
----------------------------------------


In [ ]:
def predict_emotion(text):
    preprocessed_text = preprocessor.preprocess_text(text)
    
    # Handle empty text after preprocessing
    if not preprocessed_text or len(preprocessed_text.strip()) == 0:
        return {
            'predicted_label': 'No Emotion',  # Default fallback
            'confidence': 0.0,
            'class_id': -1,
            'all_probabilities': [],
            'preprocessed_text': preprocessed_text,
            'error': 'Text became empty after preprocessing'
        }
    
    # Tokenize the preprocessed text
    inputs = emotion_tokenizer(
        preprocessed_text,  # Use preprocessed text
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    )
    
    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Run inference
    with torch.no_grad():
        outputs = emotion_model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class_id = logits.argmax(dim=-1).item()
    
    # Map back to original label
    predicted_label = emotion_label_encoder.inverse_transform([predicted_class_id])[0]
    confidence = probabilities[0][predicted_class_id].item()
    
    return {
        'predicted_label': predicted_label,
        'confidence': confidence,
        'class_id': predicted_class_id,
        'all_probabilities': probabilities[0].cpu().numpy(),
        'preprocessed_text': preprocessed_text,  # Show what the model actually saw
        'original_text': text  # Keep original for reference
    }

# Test with various types of input
test_cases = [
    "I absolutely love this product! 😍 It's amazing!",
    "This is terrible... I hate it so much 😠",
    "Check out https://example.com and @username for more info #awesome",
    "My phone just **exploded** while charging! WTF Samsung?!",
    "Meh, it's okay I guess. Nothing special.",
]

print("🎭 EMOTION PREDICTIONS:")
print("="*60)
for text in test_cases:
    result = predict_emotion(text)
    print(f"Original: {text}")
    print(f"Preprocessed: {result['preprocessed_text']}")
    print(f"Emotion: {result['predicted_label']} (confidence: {result['confidence']:.4f})")
    print("-" * 40)

🎭 EMOTION PREDICTIONS:
Original: I absolutely love this product! 😍 It's amazing!
Preprocessed: username love product ! username eye amazing !
Emotion: Fear (confidence: 0.9986)
----------------------------------------
Original: This is terrible... I hate it so much 😠
Preprocessed: username hate so much username
Emotion: Anger (confidence: 0.9975)
----------------------------------------
Original: Check out https://example.com and @username for more info #awesome
Preprocessed: check more info awesome
Emotion: Fear (confidence: 0.9981)
----------------------------------------
Original: My phone just **exploded** while charging! WTF Samsung?!
Preprocessed: phone username username ! wtf samsung ? !
Emotion: Anger (confidence: 0.9976)
----------------------------------------
Original: Meh, it's okay I guess. Nothing special.
Preprocessed: meh okay guess nothing special
Emotion: Anger (confidence: 0.9977)
----------------------------------------


In [ ]:
def predict_emotion(text):
    preprocessed_text = preprocessor.preprocess_text(text)
    
    # Handle empty text after preprocessing
    if not preprocessed_text or len(preprocessed_text.strip()) == 0:
        return {
            'predicted_label': 'No Emotion',  # Default fallback
            'confidence': 0.0,
            'class_id': -1,
            'all_probabilities': [],
            'preprocessed_text': preprocessed_text,
            'error': 'Text became empty after preprocessing'
        }
    
    # Tokenize the preprocessed text
    inputs = emotion_tokenizer(
        preprocessed_text,  # Use preprocessed text
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    )
    
    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Run inference
    with torch.no_grad():
        outputs = emotion_model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class_id = logits.argmax(dim=-1).item()
    
    # Map back to original label
    predicted_label = emotion_label_encoder.inverse_transform([predicted_class_id])[0]
    confidence = probabilities[0][predicted_class_id].item()
    
    return {
        'predicted_label': predicted_label,
        'confidence': confidence,
        'class_id': predicted_class_id,
        'all_probabilities': probabilities[0].cpu().numpy(),
        'preprocessed_text': preprocessed_text,  # Show what the model actually saw
        'original_text': text  # Keep original for reference
    }

# Test with various types of input
test_cases = [
    "I absolutely love this product! 😍 It's amazing!",
    "This is terrible... I hate it so much 😠",
    "Check out https://example.com and @username for more info #awesome",
    "My phone just **exploded** while charging! WTF Samsung?!",
    "Meh, it's okay I guess. Nothing special.",
]

print("🎭 EMOTION PREDICTIONS:")
print("="*60)
for text in test_cases:
    result = predict_emotion(text)
    print(f"Original: {text}")
    print(f"Preprocessed: {result['preprocessed_text']}")
    print(f"Emotion: {result['predicted_label']} (confidence: {result['confidence']:.4f})")
    print("-" * 40)

🎭 EMOTION PREDICTIONS:
Original: I absolutely love this product! 😍 It's amazing!
Preprocessed: username love product ! username eye amazing !
Emotion: Fear (confidence: 0.9986)
----------------------------------------
Original: This is terrible... I hate it so much 😠
Preprocessed: username hate so much username
Emotion: Anger (confidence: 0.9975)
----------------------------------------
Original: Check out https://example.com and @username for more info #awesome
Preprocessed: check more info awesome
Emotion: Fear (confidence: 0.9981)
----------------------------------------
Original: My phone just **exploded** while charging! WTF Samsung?!
Preprocessed: phone username username ! wtf samsung ? !
Emotion: Anger (confidence: 0.9976)
----------------------------------------
Original: Meh, it's okay I guess. Nothing special.
Preprocessed: meh okay guess nothing special
Emotion: Anger (confidence: 0.9977)
----------------------------------------


## Hyperparameter Tuning

In [5]:
# 1. Create the tuner for the emotion task
emotion_tuner = HyperparameterTuner(
    data_path="annotated_reddit_posts.csv",  # path to your CSV
    task_type="emotion",                     # 'emotion' column in your CSV
    n_trials=30,                             # number of Optuna trials (adjust as needed)
    cv_folds=3                               # number of cross-validation folds
)

# 2. Run the hyperparameter search
emotion_study = emotion_tuner.tune()

INFO:__main__:Loaded 95 samples for emotion task
INFO:__main__:Classes: [np.str_('Anger'), np.str_('Fear'), np.str_('Joy'), np.str_('No Emotion'), np.str_('Sadness'), np.str_('Surprise')]
INFO:__main__:Number of classes: 6
INFO:__main__:Starting hyperparameter tuning for emotion task
INFO:__main__:Search space:
INFO:__main__:  - Learning rate: [1e-5, 5e-4]
INFO:__main__:  - Batch size: [8, 16, 32]
INFO:__main__:  - Dropout rate: [0.1, 0.5]
INFO:__main__:  - Epochs: [3, 8]
INFO:__main__:  - Warmup ratio: [0.05, 0.2]
INFO:__main__:  - Weight decay: [0.01, 0.3]
[I 2025-07-09 18:05:20,294] A new study created in memory with name: no-name-afe58cd1-f687-46b4-b5f3-b75737a14e53
INFO:__main__:Trial 0: lr=4.33e-05, bs=8, dropout=0.162, epochs=3
INFO:__main__:  Fold 1/3
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifie

{'eval_loss': 1.7389585971832275, 'eval_f1': 0.07852564102564102, 'eval_runtime': 0.5467, 'eval_samples_per_second': 58.529, 'eval_steps_per_second': 7.316, 'epoch': 1.0}


                                               
 67%|██████▋   | 16/24 [00:09<00:03,  2.17it/s]

{'eval_loss': 1.6987565755844116, 'eval_f1': 0.17573529411764707, 'eval_runtime': 0.5344, 'eval_samples_per_second': 59.883, 'eval_steps_per_second': 7.485, 'epoch': 2.0}


                                               
100%|██████████| 24/24 [00:17<00:00,  2.09it/s]

...

 17%|█▋        | 4/24 [00:03<00:19,  1.05it/s]
INFO:__main__:  Fold 2/3
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\hankaixin\Desktop\multitask\env\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 17%|█▋        | 4/24 [00:03<00:15,  1.30it/s]ERROR:__main__:    Fold 2 failed: Error while serializing: IoError(Os { code: 112, kind: StorageFull, message: "There is not enough space on the disk." })


{'eval_loss': 1.798673391342163, 'eval_f1': 0.1, 'eval_runtime': 0.5359, 'eval_samples_per_second': 59.713, 'eval_steps_per_second': 3.732, 'epoch': 1.0}


 17%|█▋        | 4/24 [00:03<00:19,  1.05it/s]
INFO:__main__:  Fold 3/3
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\hankaixin\Desktop\multitask\env\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 17%|█▋        | 4/24 [00:03<00:15,  1.25it/s]ERROR:__main__:    Fold 3 failed: Error while serializing: IoError(Os { code: 112, kind: StorageFull, message: "There is not enough space on the disk." })


{'eval_loss': 1.7057788372039795, 'eval_f1': 0.0831918505942275, 'eval_runtime': 0.5224, 'eval_samples_per_second': 59.343, 'eval_steps_per_second': 3.829, 'epoch': 1.0}


 17%|█▋        | 4/24 [00:03<00:19,  1.03it/s]
INFO:__main__:Trial 29 completed: Mean F1 = 0.0000 ± 0.0000
[I 2025-07-09 20:04:02,893] Trial 29 finished with value: 0.0 and parameters: {'learning_rate': 0.00037097867093689225, 'batch_size': 16, 'dropout_rate': 0.16976717717929532, 'num_epochs': 6, 'warmup_ratio': 0.16206313991163757, 'weight_decay': 0.2553311237088122}. Best is trial 13 with value: 0.26031637717121586.
INFO:__main__:Saving optimization results...
INFO:__main__:Best score: 0.2603
INFO:__main__:Best parameters: {'learning_rate': 0.00018843871051154592, 'batch_size': 32, 'dropout_rate': 0.15859725997693935, 'num_epochs': 7, 'warmup_ratio': 0.16826426457074994, 'weight_decay': 0.21559177659152684}
INFO:__main__:Results saved to hyperparameter_tuning_emotion_20250709_180520/


In [8]:
# 3. Get the best hyperparameters found by Optuna
emotion_best_params = emotion_study.best_params

# 4. Train the final model with the best hyperparameters
emotion_model, emotion_tokenizer, emotion_encoder = train_final_model(
    emotion_best_params,
    "annotated_reddit_posts.csv",
    "emotion",
    "./roberta_emotion_model_optimized"
)

INFO:__main__:Training final emotion model with best hyperparameters...
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 21/21 [10:03<00:00, 28.75s/it]


{'train_runtime': 603.8366, 'train_samples_per_second': 1.101, 'train_steps_per_second': 0.035, 'train_loss': 1.7153429303850447, 'epoch': 7.0}


INFO:__main__:Final emotion model saved to ./roberta_emotion_model_optimized


In [9]:
print(emotion_best_params)

{'learning_rate': 0.00018843871051154592, 'batch_size': 32, 'dropout_rate': 0.15859725997693935, 'num_epochs': 7, 'warmup_ratio': 0.16826426457074994, 'weight_decay': 0.21559177659152684}


In [19]:
# Evaluate Emotion Model
print("🎭 Evaluating Emotion Model...")
emotion_results = evaluate_roberta_model(
    model_dir="./roberta_emotion_model_optimized",
    label_encoder_path="enc/emotion_label_encoder.pkl",
    csv_file="annotated_reddit_posts.csv",
    target_column="emotion",
    model_name="Emotion"
)

🎭 Evaluating Emotion Model...
🚀 Starting evaluation of Emotion model...
📂 Loading model and tokenizer...
🖥️ Using device: cuda
📊 Loading test data...
📋 Dataset info:
   - Total samples: 95
   - Classes: [np.str_('Anger'), np.str_('Fear'), np.str_('Joy'), np.str_('No Emotion'), np.str_('Sadness'), np.str_('Surprise')]
   - Class distribution:
     Anger: 24
     Fear: 4
     Joy: 12
     No Emotion: 22
     Sadness: 21
     Surprise: 12
🔮 Running model evaluation...


Predicting: 100%|██████████| 95/95 [00:01<00:00, 61.07it/s]

📈 Calculating metrics...

Emotion Model Evaluation Results:
Accuracy: 0.3579
F1-Score (Macro): 0.1613


In [ ]:
def predict_emotion(text):
    preprocessed_text = preprocessor.preprocess_text(text)
    
    # Handle empty text after preprocessing
    if not preprocessed_text or len(preprocessed_text.strip()) == 0:
        return {
            'predicted_label': 'No Emotion',  # Default fallback
            'confidence': 0.0,
            'class_id': -1,
            'all_probabilities': [],
            'preprocessed_text': preprocessed_text,
            'error': 'Text became empty after preprocessing'
        }
    
    # Tokenize the preprocessed text
    inputs = emotion_tokenizer(
        preprocessed_text,  # Use preprocessed text
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=512
    )
    
    # Move tensors to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Run inference
    with torch.no_grad():
        outputs = emotion_model(**inputs)
        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        predicted_class_id = logits.argmax(dim=-1).item()
    
    # Map back to original label
    predicted_label = emotion_label_encoder.inverse_transform([predicted_class_id])[0]
    confidence = probabilities[0][predicted_class_id].item()
    
    return {
        'predicted_label': predicted_label,
        'confidence': confidence,
        'class_id': predicted_class_id,
        'all_probabilities': probabilities[0].cpu().numpy(),
        'preprocessed_text': preprocessed_text,  # Show what the model actually saw
        'original_text': text  # Keep original for reference
    }

# Test with various types of input
test_cases = [
    "I absolutely love this product! 😍 It's amazing!",
    "This is terrible... I hate it so much 😠",
    "Check out https://example.com and @username for more info #awesome",
    "My phone just **exploded** while charging! WTF Samsung?!",
    "Meh, it's okay I guess. Nothing special.",
]

print("🎭 EMOTION PREDICTIONS:")
print("="*60)
for text in test_cases:
    result = predict_emotion(text)
    print(f"Original: {text}")
    print(f"Preprocessed: {result['preprocessed_text']}")
    print(f"Emotion: {result['predicted_label']} (confidence: {result['confidence']:.4f})")
    print("-" * 40)

🎭 EMOTION PREDICTIONS:
Original: I absolutely love this product! 😍 It's amazing!
Preprocessed: username love product ! username eye amazing !
Emotion: Fear (confidence: 0.9986)
----------------------------------------
Original: This is terrible... I hate it so much 😠
Preprocessed: username hate so much username
Emotion: Anger (confidence: 0.9975)
----------------------------------------
Original: Check out https://example.com and @username for more info #awesome
Preprocessed: check more info awesome
Emotion: Fear (confidence: 0.9981)
----------------------------------------
Original: My phone just **exploded** while charging! WTF Samsung?!
Preprocessed: phone username username ! wtf samsung ? !
Emotion: Anger (confidence: 0.9976)
----------------------------------------
Original: Meh, it's okay I guess. Nothing special.
Preprocessed: meh okay guess nothing special
Emotion: Anger (confidence: 0.9977)
----------------------------------------
